In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print("Tensorflow version: ", tf.__version__)
print("Keras version: ", keras.__version__)

Tensorflow version:  2.0.0
Keras version:  2.2.4-tf


### Load Data - Fashion MNIST

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full),(X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

class_names = ["T-shirts/Top", "Trouser", "Pullover", "Dress", 
               "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

## Model - Sequential API

In [4]:
# Model architecture
#  28x28 => flatten => 300 => relu => 100 => relu => 10 => softmax
#
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[28, 28]))
# model.add(keras.layers.Dense(300, activation="relu"))
# model.add(keras.layers.Dense(100, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

2022-09-03 09:37:46.208522: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-03 09:37:46.210557: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
             optimizer=keras.optimizers.SGD(),
             metrics=[keras.metrics.sparse_categorical_accuracy])

### Train

In [10]:
#### Custom callback: 
#### https://keras.io/guides/writing_your_own_callbacks/
class CustomCallBack(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\n val/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))
model.save("models/keras_sequential_fashion_mnist.h5")

# with call back
# checkpoint_cb = keras.callbacks.ModelCheckpoint("models/keras_sequential_fashion_mnist.h5")
# history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb])

In [9]:
model = keras.models.load_model("models/keras_sequential_fashion_mnist.h5")
X_test = X_test / 255.0
model.evaluate(X_test, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3233613259971142, 0.8825]

### Visualize Errors

In [ ]:
# Cannot load history from loaded model: 
# https://stackoverflow.com/questions/47843265/how-can-i-get-a-keras-models-history-after-loading-it-from-a-file-in-python
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

### Load Data - Housing

In [11]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [15]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

## Model - Functional API

### Multi-input, multi-output network

In [12]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [14]:
model.compile(loss={"main_output": "mse", "aux_output": "mse"}, 
              loss_weights=[0.9, 0.1], 
              optimzer=keras.optimizers.SGD)

### Train

In [17]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], 
                    epochs=10, 
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 2s 137us/sample - loss: 1.1524 - main_output_loss: 1.0771 - aux_output_loss: 1.8298 - val_loss: 0.5838 - val_main_output_loss: 0.5508 - val_aux_output_loss: 0.8797
Epoch 2/10
11610/11610 [==============================] - 1s 54us/sample - loss: 0.5121 - main_output_loss: 0.4911 - aux_output_loss: 0.6992 - val_loss: 0.4854 - val_main_output_loss: 0.4683 - val_aux_output_loss: 0.6385
Epoch 3/10
11610/11610 [==============================] - 1s 54us/sample - loss: 0.4389 - main_output_loss: 0.4149 - aux_output_loss: 0.6558 - val_loss: 0.4388 - val_main_output_loss: 0.4206 - val_aux_output_loss: 0.6025
Epoch 4/10
11610/11610 [==============================] - 1s 54us/sample - loss: 0.4054 - main_output_loss: 0.3845 - aux_output_loss: 0.5932 - val_loss: 0.4456 - val_main_output_loss: 0.4325 - val_aux_output_loss: 0.5627
Epoch 5/10
11610/11610 [==============================] - 1s 53us/

In [18]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [19]:
model.save("models/keras_functional_housing.h5")

## Model - Subclass API

In [20]:
class CustomModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = CustomModel()

In [21]:
model.compile(loss=["mse", "mse"], 
              loss_weights=[0.9, 0.1], 
              optimzer=keras.optimizers.SGD)

In [22]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], 
                    epochs=10, 
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 117us/sample - loss: 1.2243 - output_1_loss: 1.1303 - output_2_loss: 2.0672 - val_loss: 0.6062 - val_output_1_loss: 0.5625 - val_output_2_loss: 0.9993
Epoch 2/10
11610/11610 [==============================] - 1s 51us/sample - loss: 0.5734 - output_1_loss: 0.5498 - output_2_loss: 0.7845 - val_loss: 0.4970 - val_output_1_loss: 0.4762 - val_output_2_loss: 0.6836
Epoch 3/10
11610/11610 [==============================] - 1s 52us/sample - loss: 0.4647 - output_1_loss: 0.4473 - output_2_loss: 0.6201 - val_loss: 0.4530 - val_output_1_loss: 0.4359 - val_output_2_loss: 0.6055
Epoch 4/10
11610/11610 [==============================] - 1s 52us/sample - loss: 0.4201 - output_1_loss: 0.4045 - output_2_loss: 0.5602 - val_loss: 0.4418 - val_output_1_loss: 0.4260 - val_output_2_loss: 0.5830
Epoch 5/10
11610/11610 [==============================] - 1s 53us/sample - loss: 0.4070 - output_1_loss: 0

In [23]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [25]:
# You cannot save a subclassed model - use save_weights instead or upgrade to tf 2.2
# model.save("models/keras_subclass_housing.h5")